In [1]:
from pyspark.sql import SparkSession
from os.path import abspath

warehouse_location = abspath('spark-warehouse')

spark = SparkSession.builder    \
    .config('spark.driver.memory', '32g')   \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("spark.sql.catalogImplementation","hive") \
    .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:0.11.1") \
    .enableHiveSupport()   \
    .getOrCreate()
spark

:: loading settings :: url = jar:file:/workspace/apache-spark-ml/.venv/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.microsoft.azure#synapseml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5f313bad-974d-4efe-ba4b-1802937f68ff;1.0
	confs: [default]
	found com.microsoft.azure#synapseml_2.12;0.11.1 in central
	found com.microsoft.azure#synapseml-core_2.12;0.11.1 in central
	found org.scalactic#scalactic_2.12;3.2.14 in central
	found org.scala-lang#scala-reflect;2.12.15 in central
	found io.spray#spray-json_2.12;1.3.5 in central
	found com.jcraft#jsch;0.1.54 in central
	found org.apache.httpcomponents.client5#httpclient5;5.1.3 in central
	found org.apache.httpcomponents.core5#httpcore5;5.1.3 in central
	found org.apache.httpcomponents.core5#httpcore5-h2;5.1.3 in central
	found org.slf4j#slf4j-api;1.7.25 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpmime;4.5.13 in central
	found org.apache.httpcomponents#ht

In [2]:
spark.catalog.listTables()

23/06/16 05:27:45 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/06/16 05:27:45 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/06/16 05:27:47 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
23/06/16 05:27:47 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.17.0.2
23/06/16 05:27:48 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


[Table(name='data_mart', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='order_priors_prods', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='order_trains_prods', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='prd_mart', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='test_data', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='train_data', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='up_mart', catalog='spark_catalog', namespace=['default'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='user_mart', catalog='spark_catalog', namespace=

In [28]:
train_sdf = spark.table('train_data')
test_sdf = spark.table('test_data')
train_sdf.printSchema()

root
 |-- up_cnt: long (nullable = true)
 |-- up_reord_cnt: long (nullable = true)
 |-- up_no_reord_cnt: long (nullable = true)
 |-- up_reordered_avg: double (nullable = true)
 |-- up_max_ord_num: integer (nullable = true)
 |-- up_min_ord_num: integer (nullable = true)
 |-- up_avg_cart: double (nullable = true)
 |-- up_avg_prior_days: double (nullable = true)
 |-- up_max_prior_days: double (nullable = true)
 |-- up_min_prior_days: double (nullable = true)
 |-- up_avg_ord_dow: double (nullable = true)
 |-- up_avg_ord_hour: double (nullable = true)
 |-- up_usr_ratio: double (nullable = true)
 |-- up_usr_reord_ratio: double (nullable = true)
 |-- up_usr_ord_num_diff: integer (nullable = true)
 |-- usr_total_cnt: long (nullable = true)
 |-- prd_uq_cnt: long (nullable = true)
 |-- order_uq_cnt: long (nullable = true)
 |-- usr_avg_prd_cnt: double (nullable = true)
 |-- usr_avg_uq_prd_cnt: double (nullable = true)
 |-- usr_uq_prd_ratio: double (nullable = true)
 |-- usr_reord_cnt: long (nulla

In [29]:
train_sdf = train_sdf.fillna(0)

In [30]:
from pyspark.ml.feature import VectorAssembler

vector_columns = [col for col in train_sdf.columns if col != 'reordered']
vector_assembler = VectorAssembler(inputCols=vector_columns, outputCol='features')
train_sdf_vectorized = vector_assembler.transform(train_sdf)
train_sdf_vectorized.show(10)

+------+------------+---------------+------------------+--------------+--------------+------------------+------------------+-----------------+-----------------+------------------+------------------+--------------------+--------------------+-------------------+-------------+----------+------------+------------------+------------------+------------------+-------------+----------------+-------------------+------------------+------------------+------------------+------------------+-------------------------+--------------------+----------------------+-----------------+--------------------+-------------------+---------------+-------------+-------------------+------------------+------------------+------------------+----------------------+---------------+-------------------+--------------------+---------+--------------------+
|up_cnt|up_reord_cnt|up_no_reord_cnt|  up_reordered_avg|up_max_ord_num|up_min_ord_num|       up_avg_cart| up_avg_prior_days|up_max_prior_days|up_min_prior_days|    up_avg

In [31]:
train_sdf.count()

8474661

In [32]:
from synapse.ml.lightgbm import LightGBMClassifier

lgbm_estimator = LightGBMClassifier(
    labelCol='reordered', numLeaves=100,  maxDepth=10, 
    numIterations=100, earlyStoppingRound=15
)
lgbm_model = lgbm_estimator.fit(train_sdf_vectorized)

In [33]:
def make_test_sdf_ids_n_test_sdf(test_sdf):
    id_cols = ['user_id', 'product_id', 'order_id']
    test_sdf_id = test_sdf.select(*id_cols)
    test_sdf_dropped = test_sdf.drop(*id_cols)
    return test_sdf_id, test_sdf_dropped

In [34]:
test_sdf_id, test_sdf = make_test_sdf_ids_n_test_sdf(test_sdf)

In [35]:
test_sdf_id.show(10), test_sdf.show(10)

+-------+----------+--------+
|user_id|product_id|order_id|
+-------+----------+--------+
|    141|      9387|  726111|
|    141|     47766|  726111|
|    141|     10941|  726111|
|    141|     32256|  726111|
|    141|     44632|  726111|
|    141|     21137|  726111|
|    141|     31717|  726111|
|    141|     17968|  726111|
|    141|     20734|  726111|
|    141|     34448|  726111|
+-------+----------+--------+
only showing top 10 rows

+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+--------------------+-------------------+-------------------+-------------+----------+------------+---------------+------------------+------------------+-------------+----------------+-------------------+------------------+------------------+------------------+------------------+-------------------------+--------------------+----------------------+-----------------+-----

(None, None)

In [40]:
test_sdf = test_sdf.fillna(0)

In [41]:
import pyspark.sql.functions as F

test_sdf.select([F.count(F.when(F.col(c).isNull() | F.isnan(c), c)).alias(c) for c in test_sdf.columns]).show()

+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+------------+------------------+-------------------+-------------+----------+------------+---------------+------------------+----------------+-------------+----------------+-----------------+------------------+------------------+------------------+-----------------+-------------------------+--------------------+----------------------+-----------------+--------------------+-----------------+---------------+-------------+-------------+------------------+------------------+------------------+----------------------+---------------+---------------+--------------+
|up_cnt|up_reord_cnt|up_no_reord_cnt|up_reordered_avg|up_max_ord_num|up_min_ord_num|up_avg_cart|up_avg_prior_days|up_max_prior_days|up_min_prior_days|up_avg_ord_dow|up_avg_ord_hour|up_usr_ratio|up_usr_reord_ratio|up_usr_ord_num_diff|usr_total_cnt|prd_uq_

In [42]:
test_sdf_vectorized = vector_assembler.transform(test_sdf)
test_sdf_vectorized.show(10)
predictions = lgbm_model.transform(test_sdf_vectorized)
predictions.show(10)

+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+--------------------+-------------------+-------------------+-------------+----------+------------+---------------+------------------+------------------+-------------+----------------+-------------------+------------------+------------------+------------------+------------------+-------------------------+--------------------+----------------------+-----------------+--------------------+-------------------+---------------+-------------+-------------------+------------------+------------------+------------------+----------------------+---------------+-------------------+-------------------+--------------------+
|up_cnt|up_reord_cnt|up_no_reord_cnt|up_reordered_avg|up_max_ord_num|up_min_ord_num|up_avg_cart|up_avg_prior_days|up_max_prior_days|up_min_prior_days|up_avg_ord_dow|up_avg_ord_hour|        up_usr_ratio|

23/06/16 05:52:27 WARN DAGScheduler: Broadcasting large task binary with size 1226.4 KiB


+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+--------------------+-------------------+-------------------+-------------+----------+------------+---------------+------------------+------------------+-------------+----------------+-------------------+------------------+------------------+------------------+------------------+-------------------------+--------------------+----------------------+-----------------+--------------------+-------------------+---------------+-------------+-------------------+------------------+------------------+------------------+----------------------+---------------+-------------------+-------------------+--------------------+--------------------+--------------------+----------+
|up_cnt|up_reord_cnt|up_no_reord_cnt|up_reordered_avg|up_max_ord_num|up_min_ord_num|up_avg_cart|up_avg_prior_days|up_max_prior_days|up_min_prior_days

In [44]:
def make_predictions_with_ids(predictions, sdf_id):
    sdf_ids_with_row_id = sdf_id.withColumn('row_id', F.monotonically_increasing_id())
    predictions_with_row_id = predictions.withColumn('row_id', F.monotonically_increasing_id())
    return sdf_ids_with_row_id.join(predictions_with_row_id, 'row_id').drop('row_id')

In [45]:
predictions = make_predictions_with_ids(predictions, test_sdf_id)
predictions.show(10)

23/06/16 05:56:06 WARN DAGScheduler: Broadcasting large task binary with size 1236.8 KiB
23/06/16 05:56:29 WARN DAGScheduler: Broadcasting large task binary with size 1259.0 KiB


+-------+----------+--------+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+--------------------+--------------------+-------------------+-------------+----------+------------+------------------+------------------+------------------+-------------+----------------+-------------------+------------------+------------------+------------------+------------------+-------------------------+--------------------+----------------------+-----------------+--------------------+-------------------+---------------+-------------+-------------------+------------------+------------------+------------------+----------------------+---------------+-------------------+--------------------+--------------------+--------------------+--------------------+----------+
|user_id|product_id|order_id|up_cnt|up_reord_cnt|up_no_reord_cnt|up_reordered_avg|up_max_ord_num|up_min_ord_num|up_a

In [46]:
from pyspark.ml.functions import vector_to_array

def make_reordered_with_threshold(predictions, threshold=0.21):
    predictions = predictions.withColumn('probability_arr', vector_to_array('probability')) \
                                .withColumn('1_proba', F.col('probability_arr')[1]) \
                                .withColumn('reordered', (F.col('1_proba') > threshold).cast('int'))
    return predictions


In [47]:
predictions = make_reordered_with_threshold(predictions)
predictions.show(10)

23/06/16 06:00:32 WARN DAGScheduler: Broadcasting large task binary with size 1236.8 KiB
23/06/16 06:00:51 WARN DAGScheduler: Broadcasting large task binary with size 1276.7 KiB


+-------+----------+--------+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+--------------------+--------------------+-------------------+-------------+----------+------------+------------------+------------------+------------------+-------------+----------------+-------------------+------------------+------------------+------------------+------------------+-------------------------+--------------------+----------------------+-----------------+--------------------+-------------------+---------------+-------------+-------------------+------------------+------------------+------------------+----------------------+---------------+-------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+-------------------+---------+
|user_id|product_id|order_id|up_cnt|up_reord_cnt|up_no_reord_cnt|

In [56]:
def make_submission(predictions):
    predictions_grp = predictions.groupby('order_id').agg(
        F.count('*').alias('total_cnt_by_order_id'),
        F.sum(F.col('reordered')).alias('reordered_cnt')
    )
    udf_get_product_ids_str = F.udf(lambda product_id_group: ' '.join(map(str, product_id_group)))
    submission_01 = predictions.filter(F.col('reordered') == 1).groupby('order_id').agg(
        udf_get_product_ids_str(F.collect_list('product_id')).alias('products')
    )
    submission_02 = predictions_grp.filter(F.col('reordered_cnt') == 0)  \
                                    .withColumn('products', F.lit('None'))  \
                                    .select('order_id', 'products')
    submission = submission_01.union(submission_02)
    return submission

In [57]:
submission = make_submission(predictions)
submission.show(10)

23/06/16 06:26:07 WARN DAGScheduler: Broadcasting large task binary with size 1216.7 KiB
23/06/16 06:26:20 WARN DAGScheduler: Broadcasting large task binary with size 1210.8 KiB
23/06/16 06:26:36 WARN DAGScheduler: Broadcasting large task binary with size 1200.7 KiB
23/06/16 06:26:37 WARN DAGScheduler: Broadcasting large task binary with size 1222.2 KiB


+--------+--------------------+
|order_id|            products|
+--------+--------------------+
|      34|39180 39475 2596 ...|
|     137|5134 2326 23794 2...|
|     353|33000 21137 40688...|
|     386|21479 42265 21903...|
|     497|39947 31964 27275...|
|     604|13176 19660 24852...|
|     758|           651 19660|
|    1304|42585 42265 24852...|
|    1802|17830 31612 33043...|
|    2247|33198 19125 13176...|
+--------+--------------------+
only showing top 10 rows



23/06/16 06:26:38 WARN DAGScheduler: Broadcasting large task binary with size 1282.7 KiB


In [81]:
train_sdf = spark.table('train_data')
test_sdf = spark.table('test_data')
train_sdf = train_sdf.fillna(0)
tr_sdf, val_sdf = train_sdf.randomSplit([0.8, 0.2])
tr_sdf = tr_sdf.withColumn('isVal', F.lit(False))
val_sdf = val_sdf.withColumn('isVal', F.lit(True))
tr_val_sdf = tr_sdf.union(val_sdf)
tr_val_sdf.show(10)
train_sdf.count(), tr_sdf.count(), val_sdf.count(), tr_val_sdf.count()

+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+--------------------+------------------+-------------------+-------------+----------+------------+------------------+------------------+-------------------+-------------+----------------+-------------------+------------------+------------------+------------------+------------------+-------------------------+--------------------+----------------------+-----------------+--------------------+------------------+---------------+-------------+-------------------+------------------+------------------+------------------+----------------------+---------------+-------------------+-------------------+---------+-----+
|up_cnt|up_reord_cnt|up_no_reord_cnt|up_reordered_avg|up_max_ord_num|up_min_ord_num|up_avg_cart|up_avg_prior_days|up_max_prior_days|up_min_prior_days|up_avg_ord_dow|up_avg_ord_hour|        up_usr_ratio|up_

(8474661, 6778222, 1696439, 8474661)

In [72]:
def make_test_sdf_ids_n_test_sdf(test_sdf):
    id_cols = ['user_id', 'product_id', 'order_id']
    test_sdf_id = test_sdf.select(*id_cols)
    test_sdf_dropped = test_sdf.drop(*id_cols)
    return test_sdf_id, test_sdf_dropped

def make_predictions_with_ids(predictions, sdf_id):
    sdf_ids_with_row_id = sdf_id.withColumn('row_id', F.monotonically_increasing_id())
    predictions_with_row_id = predictions.withColumn('row_id', F.monotonically_increasing_id())
    return sdf_ids_with_row_id.join(predictions_with_row_id, 'row_id').drop('row_id')

def make_reordered_with_threshold(predictions, label='reordered', col_type='int', threshold=0.21):
    predictions = predictions.withColumn('probability_arr', vector_to_array('probability')) \
                                .withColumn('1_proba', F.col('probability_arr')[1]) \
                                .withColumn(label, (F.col('1_proba') > threshold).cast(col_type))
    return predictions

def make_submission(predictions):
    predictions_grp = predictions.groupby('order_id').agg(
        F.count('*').alias('total_cnt_by_order_id'),
        F.sum(F.col('reordered')).alias('reordered_cnt')
    )
    udf_get_product_ids_str = F.udf(lambda product_id_group: ' '.join(map(str, product_id_group)))
    submission_01 = predictions.filter(F.col('reordered') == 1).groupby('order_id').agg(
        udf_get_product_ids_str(F.collect_list('product_id')).alias('products')
    )
    submission_02 = predictions_grp.filter(F.col('reordered_cnt') == 0)  \
                                    .withColumn('products', F.lit('None'))  \
                                    .select('order_id', 'products')
    submission = submission_01.union(submission_02)
    return submission

In [60]:
from pyspark.ml.feature import VectorAssembler
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from synapse.ml.lightgbm import LightGBMClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

search_space = {
    'maxDepth': hp.quniform('maxDepth', 10, 20, 1),
    'numLeaves': hp.quniform('numLeaves', 32, 78, 4),
    'baggingFraction': hp.uniform('baggingFraction', 0.6, 1.0),
    'featureFraction': hp.uniform('featureFraction', 0.6, 1.0),
    'learningRate': hp.uniform('learningRate', 0.01, 0.1),
}
vector_columns = [column_name for column_name in train_sdf.columns if column_name != 'reordered']
vector_assembler = VectorAssembler(inputCols=vector_columns, outputCol='features')
tr_sdf_vectorized = vector_assembler.transform(tr_sdf)
val_sdf_vectorized = vector_assembler.transform(val_sdf)
f1_eval = MulticlassClassificationEvaluator(labelCol='reordered')

In [66]:
def objective_func(space):
    lgbm_classifier = LightGBMClassifier(
        numIterations=50, 
        maxDepth=int(space['maxDepth']),
        numLeaves=int(space['numLeaves']),
        baggingFraction=space['baggingFraction'],
        featureFraction=space['featureFraction'],
        learningRate=space['learningRate'],
        featuresCol='features',
        labelCol='reordered'
    )
    lgbm_model = lgbm_classifier.fit(tr_sdf_vectorized)
    predictions = lgbm_model.transform(val_sdf_vectorized)
    predictions = make_reordered_with_threshold(predictions, label='prediction', col_type='double')
    f1_eval = MulticlassClassificationEvaluator(labelCol='reordered')
    f1_score = f1_eval.evaluate(predictions)
    print('space:', space, 'f1_score:', f1_score)
    return {'loss': -1*f1_score, 'status': STATUS_OK}

In [67]:
algo = tpe.suggest
trials = Trials()
best = fmin(fn=objective_func, space=search_space, max_evals=25, algo=algo, trials=trials)

  0%|          | 0/25 [00:00<?, ?trial/s, best loss=?]

space:                                                
{'baggingFraction': 0.7523826538313113, 'featureFraction': 0.6644398196577093, 'learningRate': 0.01867697015202177, 'maxDepth': 13.0, 'numLeaves': 56.0}
f1_score:                                             
0.8908494032123885                                    
  4%|▍         | 1/25 [00:52<20:59, 52.49s/trial, best loss: -0.8908494032123885]

space:                                                                           
{'baggingFraction': 0.6272292951923831, 'featureFraction': 0.618365626486771, 'learningRate': 0.08248127898357833, 'maxDepth': 13.0, 'numLeaves': 32.0}
f1_score:                                                                        
0.8801132625400485                                                               
  8%|▊         | 2/25 [01:42<19:36, 51.16s/trial, best loss: -0.8908494032123885]

space:                                                                           
{'baggingFraction': 0.6552312083139652, 'featureFraction': 0.9181954119235816, 'learningRate': 0.0867127892714699, 'maxDepth': 17.0, 'numLeaves': 44.0}
f1_score:                                                                        
0.8799800203635076                                                               
 12%|█▏        | 3/25 [02:35<19:05, 52.05s/trial, best loss: -0.8908494032123885]

space:                                                                           
{'baggingFraction': 0.8330794216351993, 'featureFraction': 0.751032552297082, 'learningRate': 0.04255464263744633, 'maxDepth': 11.0, 'numLeaves': 56.0}
f1_score:                                                                        
0.882928884226569                                                                
 16%|█▌        | 4/25 [03:31<18:45, 53.57s/trial, best loss: -0.8908494032123885]

...

space:                                                                            
{'baggingFraction': 0.9899423025141096, 'featureFraction': 0.6555093885155099, 'learningRate': 0.010129679035660458, 'maxDepth': 10.0, 'numLeaves': 64.0}
f1_score:                                                                         
0.8895538667336589                                                                
 84%|████████▍ | 21/25 [18:36<03:30, 52.59s/trial, best loss: -0.8908494032123885]

space:                                                                            
{'baggingFraction': 0.9990850277937957, 'featureFraction': 0.6074630864058319, 'learningRate': 0.010773107205551734, 'maxDepth': 11.0, 'numLeaves': 64.0}
f1_score:                                                                         
0.8901524485100457                                                                
 88%|████████▊ | 22/25 [19:27<02:36, 52.26s/trial, best loss: -0.8908494032123885]

space:                                                                            
{'baggingFraction': 0.6779798216518752, 'featureFraction': 0.611292153423481, 'learningRate': 0.010353110067299644, 'maxDepth': 11.0, 'numLeaves': 76.0}
f1_score:                                                                         
0.8899294084168872                                                                
 92%|█████████▏| 23/25 [20:21<01:45, 52.55s/trial, best loss: -0.8908494032123885]

space:                                                                            
{'baggingFraction': 0.9536569345860515, 'featureFraction': 0.7053638044032232, 'learningRate': 0.017661750965652052, 'maxDepth': 11.0, 'numLeaves': 64.0}
f1_score:                                                                         
0.8911706398136663                                                                
 96%|█████████▌| 24/25 [21:13<00:52, 52.65s/trial, best loss: -0.8911706398136663]

space:                                                                            
{'baggingFraction': 0.9618616429017719, 'featureFraction': 0.7113274159882055, 'learningRate': 0.020850854970521544, 'maxDepth': 12.0, 'numLeaves': 60.0}
f1_score:                                                                         
0.8896441492674925                                                                
100%|██████████| 25/25 [22:06<00:00, 53.08s/trial, best loss: -0.8911706398136663]


In [73]:
print('best:', best)

best: {'baggingFraction': 0.9536569345860515, 'featureFraction': 0.7053638044032232, 'learningRate': 0.017661750965652052, 'maxDepth': 11.0, 'numLeaves': 64.0}


In [82]:
train_sdf_vectorized = vector_assembler.transform(train_sdf)
lgbm_classifier = LightGBMClassifier(
        numIterations=300, 
        maxDepth=int(best['maxDepth']),
        numLeaves=int(best['numLeaves']),
        baggingFraction=best['baggingFraction'],
        featureFraction=best['featureFraction'],
        learningRate=best['learningRate'],
        featuresCol='features',
        labelCol='reordered'
    )
lgbm_model = lgbm_estimator.fit(train_sdf_vectorized)
test_sdf_id, test_sdf = make_test_sdf_ids_n_test_sdf(test_sdf)
test_sdf = test_sdf.fillna(0)
test_sdf_vectorized = vector_assembler.transform(test_sdf)
predictions = lgbm_model.transform(test_sdf_vectorized)

In [83]:
predictions = make_predictions_with_ids(predictions, test_sdf_id)
predictions.show(10)

23/06/16 09:15:10 WARN DAGScheduler: Broadcasting large task binary with size 1236.8 KiB
23/06/16 09:15:28 WARN DAGScheduler: Broadcasting large task binary with size 1259.0 KiB


+-------+----------+--------+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+--------------------+--------------------+-------------------+-------------+----------+------------+------------------+------------------+------------------+-------------+----------------+-------------------+------------------+------------------+------------------+------------------+-------------------------+--------------------+----------------------+-----------------+--------------------+-------------------+---------------+-------------+-------------------+------------------+------------------+------------------+----------------------+---------------+-------------------+--------------------+--------------------+--------------------+--------------------+----------+
|user_id|product_id|order_id|up_cnt|up_reord_cnt|up_no_reord_cnt|up_reordered_avg|up_max_ord_num|up_min_ord_num|up_a

In [84]:
predictions = make_reordered_with_threshold(predictions)
predictions.show(10)

23/06/16 09:15:33 WARN DAGScheduler: Broadcasting large task binary with size 1236.8 KiB
23/06/16 09:15:52 WARN DAGScheduler: Broadcasting large task binary with size 1276.7 KiB


+-------+----------+--------+------+------------+---------------+----------------+--------------+--------------+-----------+-----------------+-----------------+-----------------+--------------+---------------+--------------------+--------------------+-------------------+-------------+----------+------------+------------------+------------------+------------------+-------------+----------------+-------------------+------------------+------------------+------------------+------------------+-------------------------+--------------------+----------------------+-----------------+--------------------+-------------------+---------------+-------------+-------------------+------------------+------------------+------------------+----------------------+---------------+-------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+-------------------+---------+
|user_id|product_id|order_id|up_cnt|up_reord_cnt|up_no_reord_cnt|

In [85]:
submission = make_submission(predictions)

In [86]:
submission.coalesce(1).write.csv('data/instacart_market/submission_02', header=True)

23/06/16 09:16:06 WARN DAGScheduler: Broadcasting large task binary with size 1216.7 KiB
23/06/16 09:16:36 WARN DAGScheduler: Broadcasting large task binary with size 1210.9 KiB
23/06/16 09:17:06 WARN DAGScheduler: Broadcasting large task binary with size 1200.7 KiB
23/06/16 09:17:08 WARN DAGScheduler: Broadcasting large task binary with size 1222.2 KiB
23/06/16 09:17:10 WARN DAGScheduler: Broadcasting large task binary with size 1521.9 KiB
